In [7]:
##%overwritefile
##%file:src/do_execute.py
##%noruncode
    def do_execute(self, code, silent, store_history=True,
                   user_expressions=None, allow_stdin=True):
        try:
            self.silent = silent
            magics, code = self._filter_magics(code)
            if len(magics['replcmdmode'])>0:
                return self.send_replcmd(code, silent, store_history=True,
                    user_expressions=None, allow_stdin=False)
            if len(magics['noruncode'])>0 and ( len(magics['command'])>0 or len(magics['pythoncmd'])>0):
                return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            # if len(magics['file'])<1:
            #     magics, code = self._add_main(magics, code)
            ##
            ##调用接口
            ##self.callISp_before_exec(code)
            with self.new_temp_file(suffix='.py',dir=os.path.abspath('')) as source_file:
                source_file.write(code)
                source_file.flush()
                newsrcfilename=source_file.name
                if len(magics['file'])>0:
                    newsrcfilename = self._fileshander(magics['file'],newsrcfilename,magics)
                    # newsrcfilename = magics['file'][0]
                    # newsrcfilename = os.path.join(os.path.abspath(''),newsrcfilename)
                    # if os.path.exists(newsrcfilename):
                    #     if len(magics['overwritefile'])<1:
                    #         newsrcfilename +=".new.py"
                    # if not os.path.exists(os.path.dirname(newsrcfilename)) :
                    #     os.makedirs(os.path.dirname(newsrcfilename))
                    # os.rename(source_file.name,newsrcfilename)

                    self._write_to_stdout("[MyPythonkernel] Info:file "+ newsrcfilename +" created successfully\n")
            if len(magics['noruncode'])>0:
                return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}
            ##
            ##调用接口
            ## self.callIBp_before_exec(code)
            
            self._write_to_stdout("The process :"+newsrcfilename+"\n")
            p = self.create_jupyter_subprocess(['python',newsrcfilename]+ magics['args'],cwd=None,shell=False,env=magics['env'])
            #p = self.create_jupyter_subprocess([binary_file.name]+ magics['args'],cwd=None,shell=False)
            #p = self.create_jupyter_subprocess([self.master_path, binary_file.name] + magics['args'],cwd='/tmp',shell=True)
            self.g_rtsps[str(p.pid)]=p
            if len(magics['showpid'])>0:
                self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
            while p.poll() is None:
                p.write_contents(magics)
            self._write_to_stdout("The process end:"+str(p.pid)+"\n")
            # wait for threads to finish, so output is always shown
            p._stdout_thread.join()
            p._stderr_thread.join()
            # del self.g_rtsps[str(p.pid)]
            p.write_contents(magics)
            ##
            ##调用接口
            ##self.callIBp_after_exec(p.returncode)
            ##
            ##调用接口
            ## self.callISp_after_exec(p.returncode)
            self.cleanup_files()
            if p.returncode != 0:
                self._write_to_stderr("[MyPythonkernel] Executable exited with code {}".format(p.returncode))
        except Exception as e:
            self._write_to_stderr("[MyPythonkernel] Error "+str(e))
        return {'status': 'ok', 'execution_count': self.execution_count, 'payload': [], 'user_expressions': {}}


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/do_execute.py created successfully


In [5]:
##%overwritefile
##%file:../jupyter-MyPython-kernel/jupyter_MyPython_kernel/kernel.py
##//%file:kernel.py
##%noruncode
#
#   MyPython Jupyter Kernel
#
##%include:src/head.py

##%include:src/common.py



##%include:src/IREPLWrapper.py
##%include:src/RealTimeSubprocess.py

class MyPythonKernel(Kernel):
    implementation = 'jupyter-MyPython-kernel'
    implementation_version = '1.0'
    language = 'Python'
    language_version = sys.version.split()[0]
    language_info = {'name': 'python',
                     'version': sys.version.split()[0],
                     'mimetype': 'text/x-python',
                     'codemirror_mode': {
                        'name': 'ipython',
                        'version': sys.version_info[0]
                     },
                     'pygments_lexer': 'ipython%d' % 3,
                     'nbconvert_exporter': 'python',
                     'file_extension': '.py'}
    banner = "MyPython kernel.\n" \
             "Uses gcc, compiles in C11, and creates source code files and executables in temporary folder.\n"
    kernelinfo="[MyPython]"
    main_head = "\n" \
            "\n" \
            "int main(List<String> arguments){\n"

    main_foot = "\nreturn 0;\n}"

##%include:src/comm_attribute.py

    def __init__(self, *args, **kwargs):
        super(MyPythonKernel, self).__init__(*args, **kwargs)
        self._allow_stdin = True
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
        self.files = []
        self.resDir = path.join(path.dirname(path.realpath(__file__)), 'resources')

        self.chk_replexit_thread = Thread(target=self.chk_replexit, args=(self.g_rtsps))
        self.chk_replexit_thread.daemon = True
        self.chk_replexit_thread.start()

##%include:src/_templateHander.py
##%include:src/_readtemplatefile.py

##%include:src/cleannotes.py
##%include:src/kernel_common.py
##%include:src/_is_specialID.py
##%include:src/repl_listpid.py
##%include:src/chk_replexit.py
##%include:src/cleanup_files.py
##%include:src/new_temp_file.py
##%include:src/log_output.py
##%include:src/_write_display_data.py
##%include:src/_write_to_stdout.py
##%include:src/_write_to_stderr.py
##%include:src/_read_from_stdin.py
##%include:src/readcodefile.py
    #####################################################################
##%include:src/_start_replprg.py
##%include:src/process_output.py
##%include:src/send_replcmd.py
    #####################################################################
##%include:src/do_shell_command.py
    
##%include:src/do_Py_command.py
    
##%include:src/do_flutter_command.py
    
##%include:src/send_cmd.py
##%include:src/create_jupyter_subprocess.py
##%include:src/generate_Pythonfile.py
##%include:src/compile_with_Python2native.py0
##%include:src/_filter_env.py
#####################################################################
##%include:src/py_filter_magics.py
#####################################################################
##%include:src/_add_main.py
##%include:src/do_execute.py
##%include:src/do_shutdown.py
#####################################################################
##%include:src/pluginmng.py

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/../jupyter-MyPython-kernel/jupyter_MyPython_kernel/kernel.py created successfully
